PREREQUISITE: Make sure thanhhoa_processed.csv is located in data/processed. If not, run script files

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

path = "../data/processed/thanhhoa_processed.csv"
df = read_csv(path)

df.head()